# Databricks Managed MCP Server for Genie, Vector Search, and Custom Functions

In [0]:
%pip install -U "mcp>=1.9" "databricks-sdk[openai]" "mlflow>=3.1.0" "databricks-agents>=1.0.0" "databricks-mcp" databricks_ai_bridge

dbutils.library.restartPython()

## MCP with Genie

In [0]:
space_id = "01f05e591d711e81bd981dae9784abbd"

# Initialize the Databricks MCP client with the specified server URL and workspace client
mcp_client = DatabricksMCPClient(
    server_url=f"https:///adb-984752964297111.11.azuredatabricks.net/api/2.0/mcp/genie/{space_id}",
    workspace_client=workspace_client,
)

# Access the genie client from the MCP client
genie = mcp_client.client.genie

# Start a conversation with the genie and wait for a response
response = genie.start_conversation_and_wait(
    space_id=space_id,
    content="what are different type of order status?",
)

# Retrieve the query result from the message attachment
genie.get_message_attachment_query_result(
    space_id=space_id,
    message_id=response.message_id,
    conversation_id=response.conversation_id,
    attachment_id=response.attachments[0].attachment_id,
).statement_response.result.data_array,

## MCP with Databricks Vector Search

In [0]:
catalog_name = "mc"
schema_name = "fidel"

# Initialize the Databricks MCP client with the specified server URL and workspace client
mcp_client = DatabricksMCPClient(
    server_url=f"https:///adb-984752964297111.11.azuredatabricks.net/api/2.0/mcp/vector-search/{catalog_name}/{schema_name}",
    workspace_client=workspace_client,
)

# Query the vector search index with the specified index name and query text
mcp_client.client.vector_search_indexes.query_index(
    index_name="mc.fidel.vs_index",
    query_text="RAG",
    columns=["page_content_chunk"],
).result.data_array

## MCP with Custom Functions

In [0]:
import nest_asyncio
import pandas as pd
import json
from databricks_mcp import DatabricksMCPClient
from databricks.sdk import WorkspaceClient

workspace_client = WorkspaceClient()
mcp_server_url = (
    "https://adb-984752964297111.11.azuredatabricks.net/api/2.0/mcp/functions/system/ai"
)
nest_asyncio.apply()

code = """print("Hello, world!")"""


def test_connect_to_server(code):
    mcp_client = DatabricksMCPClient(
        server_url=mcp_server_url, workspace_client=workspace_client
    )

    tools = mcp_client.list_tools()
    print(f"Discovered tools {[t.name for t in tools]}")

    result = mcp_client.call_tool("system__ai__python_exec", {"code": code})
    return result.content[0].text


# display(pd.DataFrame([test_connect_to_server(code)]))

result = test_connect_to_server(code)

display(pd.DataFrame((json.loads(result))))